In [1]:
import os

if os.path.isdir("../notebooks/"):
    os.chdir("..")

In [2]:
import json
import random
import itertools
import collections

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gensim.models as gm
from tqdm import tqdm
from gensim.models import KeyedVectors
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity


from badseeds import replicate_fig2
from badseeds import utils, seedbank, metrics, replicate_bolukbasi, replicate_fig4

In [3]:
# path to config json file containing paths to datasets. change if necessary
CONFIG_PATH = "./config.json"

In [4]:
with open(CONFIG_PATH, "r") as f:
    config = json.load(f)

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns

plt.rcParams["font.family"] = "serif"

In [6]:
# for replicability
np.random.seed(42)
random.seed(42)

## Models and Data

### Models

In [7]:
models = []
# load google news word2vec
# Load vectors directly from the file
# models.append(
#     KeyedVectors.load_word2vec_format(
#         os.path.join(
#             config["models"]["dir_path"], config["models"]["google_news_subpath"]
#         )
#         + ".bin",
#         binary=True,
#     )
# )

# replicate fig. 3 with NYT dataset
# get embeddings trained on NYT with min freq of 100

direct = os.fsencode(
    os.path.join(config["models"]["dir_path"], config["models"]["nyt_subpath"]["100"])
)

for filename in os.listdir(direct):
    f = os.path.join(direct, filename)

    # checking if it is a file
    if os.path.isfile(f):
        f = os.fsdecode(f)
        models.append(KeyedVectors.load(f))

### Seeds

In [8]:
# get desired seeds:
seed = seedbank.seedbanking(config["seeds"]["dir_path"] + "seeds.json", index="ID")

gender_seed_list = [
    "definitional_female-Bolukbasi_et_al_2016",
    "definitional_male-Bolukbasi_et_al_2016",
]

# lower case seeds? she didnt do it in appendix (doesnt make sense tho)

seed_list = [seed.loc[seed_set]["Seeds"] for seed_set in gender_seed_list]
seed1 = seed_list[0]
seed2 = seed_list[1]

# hard coded shuffled seeds from paper
seed1_shuf = [
    "female",
    "she",
    "woman",
    "gal",
    "her",
    "daughter",
    "girl",
    "herself",
    "mother",
    "mary",
]
# misses seed
seed2_shuf = [
    "john",
    "man",
    "son",
    "father",
    "male",
    "himself",
    "guy",
    "he",
    "his",
    "boy",
]

seed2_rnd = [
    "chun",
    "brush",
    "dictates",
    "caesar",
    "fewest",
    "breitbart",
    "rod",
    "heaped",
    "julianna",
    "longest",
]
seed1_rnd = [
    "negatives",
    "vel",
    "theirs",
    "canoe",
    "meet",
    "bilingual",
    "mor",
    "facets",
    "fari",
    "lily",
]

NameError: name 'clean' is not defined

### Function call

In [ ]:
(
    variance_ordered,
    variance_rnd,
    variance_inshuffle,
) = replicate_bolukbasi.pca_seeds_model(
    seed1, seed2, models, seed1_shuf=seed1_shuf, seed2_shuf=seed2_shuf, components=True
)
print(variance_rnd.shape)

#### Get top 10 and bottom 10 highest ranked words, by cosine similarity

In [ ]:
(
    gender_pairs_values, 
    gender_pairs_words, 
    random_pairs_values, 
    random_pairs_words, 
    shuffled_gender_pairs_values, 
    shuffled_gender_pairs_words,
) = replicate_fig4.figure_4(variance_ordered, variance_rnd, variance_inshuffle)


## Visualization

In [ ]:
# plot
fig, (ax1, ax2, ax3) = plt.subplots(1, 3)
fig.set_size_inches(w=6.50127, h=5)
fig.tight_layout(rect=[0, 0, 0.9, 1], pad=6)

all_values = np.concatenate(
    [gender_pairs_values, random_pairs_values, shuffled_gender_pairs_values]
)
vmin, vmax = np.min(all_values), np.max(all_values)

ax1 = sns.heatmap(
    gender_pairs_values[::-1, np.newaxis],
    yticklabels=gender_pairs_words,
    xticklabels=[],
    cmap=plt.get_cmap("PiYG"),
    ax=ax1,
    cbar=False,
    annot=True,
    center=np.zeros(1),
    linewidths=0.5,
    vmin=vmin,
    vmax=vmax,
)
ax1.set_yticklabels(ax1.get_yticklabels(), rotation=0, fontsize=10)
ax1.set_xlabel("gender word \n pairs")

ax2 = sns.heatmap(
    random_pairs_values[::-1, np.newaxis],
    yticklabels=random_pairs_words,
    xticklabels=[],
    cmap=plt.get_cmap("PiYG"),
    ax=ax2,
    cbar=False,
    annot=True,
    center=0,
    linewidths=0.5,
    vmin=vmin,
    vmax=vmax,
)
ax2.set_yticklabels(ax2.get_yticklabels(), rotation=0, fontsize=10)
ax2.set_xlabel("random word \n pairs")

ax3 = sns.heatmap(
    shuffled_gender_pairs_values[::-1, np.newaxis],
    yticklabels=shuffled_gender_pairs_words,
    xticklabels=[],
    cmap=plt.get_cmap("PiYG"),
    ax=ax3,
    cbar=False,
    annot=True,
    center=0,
    linewidths=0.5,
    vmin=vmin,
    vmax=vmax,
)
ax3.set_yticklabels(ax3.get_yticklabels(), rotation=0, fontsize=10)
ax3.set_xlabel("shuffled gender \n word pairs")

plt.show()

In [ ]:
# save to latex
plt.rc("pgf", texsystem="pdflatex")
fig.savefig("images/fig_4.pdf", bbox_inches="tight")

##### Optional: use top 10 components of first model and ranks them across all models

In [ ]:
vals = []
words = []
# for var in [variance_ordered,variance_rnd,variance_inshuffle]:
temp = np.mean(variance_ordered, axis=0)[0]
cos_sim = cosine_similarity(overlap_embed, [temp]).flatten()
v, w = zip(*sorted(zip(list(cos_sim), overlap_list)))
vals.append(v[:10] + v[-10:])
words.append((w[:10] + w[-10:]))
word = w[:10] + w[-10:]


temp = np.mean(variance_rnd, axis=0)[0]
cos_sim = cosine_similarity(overlap_embed, [temp]).flatten()
v, w = zip(*sorted(zip(list(cos_sim), word)))
vals.append(v[:10] + v[-10:])
words.append((w[:10] + w[-10:]))

temp = np.mean(variance_inshuffle, axis=0)[0]
cos_sim = cosine_similarity(overlap_embed, [temp]).flatten()
v, w = zip(*sorted(zip(list(cos_sim), word)))
vals.append(v[:10] + v[-10:])
words.append((w[:10] + w[-10:]))

gender_pairs_values, gender_pairs_words = np.asarray(vals[0]), words[0]
random_pairs_values, random_pairs_words = np.asarray(vals[1]), words[1]
shuffled_gender_pairs_values, shuffled_gender_pairs_words = (
    np.asarray(vals[2]),
    words[2],
)

In [ ]:
fig, (ax) = plt.subplots()
fig.set_size_inches(w=0.7, h=5)
ax = sns.heatmap(
    shuffled_gender_pairs_values[::-1, np.newaxis],
    yticklabels=shuffled_gender_pairs_words,
    xticklabels=[],
    cmap=plt.get_cmap("PiYG"),
    #     ax=ax3,
    cbar=False,
    annot=True,
    center=0,
    linewidths=0.5,
    vmin=-0.62,
    vmax=0.62,
)
ax.set_yticklabels(ax.get_yticklabels(), rotation=0, fontsize=10)
ax.set_xlabel("shuffled gender \n word pairs")
plt.show()

In [ ]:
# save to latex
plt.rc("pgf", texsystem="pdflatex")
fig.savefig("images/fig_4_2.pdf", bbox_inches="tight")